In [30]:
import re

class DataExtractor:
    def __init__(self, data):
        self.data = data

    def extract_units(self):
        # Regular expression to match patterns like "años", "meses", "días"
        unit_pattern = re.compile(r'\b(\d+)\s*(años|meses|días|horas)\b|\b(años|meses|días|horas)\s*(\d+)\b')
        matches = unit_pattern.findall(self.data)
        
        units = []
        for match in matches:
            if match[0] and match[1]:
                units.append((match[0], match[1]))
            elif match[2] and match[3]:
                units.append((match[3], match[2]))
        
        return units

# Ejemplo de uso
data = "Menarca a los 14 años"
extractor = DataExtractor(data)
unidades = extractor.extract_units()
for unidad, tipo_unidad in unidades:
    print(f"Unidad: {unidad}, Tipo de unidad: {tipo_unidad}")

Unidad: 14, Tipo de unidad: años


In [32]:
import re
import os

class LimpiezaTexto:
    def __init__(self, nombreArchivo):
        self.nombreArchivo = nombreArchivo
        self.informacion = None

        # Cargar datos al inicializar el objeto
        self.cargar_datos()
        self.conversion_minusculas()
        self.eliminar_caracteres_iniciales()

    def cargar_datos(self):
        if not os.path.exists(self.nombreArchivo):
            raise FileNotFoundError(f"El archivo {self.nombreArchivo} no existe.")
        with open(self.nombreArchivo, 'r', encoding='utf-8') as file:
            self.informacion = [linea.strip() for linea in file if linea.strip()]

    def conversion_minusculas(self):
        self.informacion = [linea.lower() for linea in self.informacion]

    def eliminar_caracteres_iniciales(self):
        self.informacion = [re.sub(r'^[\s]*[^a-zA-Z0-9]+', '', linea) for linea in self.informacion]

    
    def preprocesar_texto(self):
        # Lista de claves conocidas
        claves = [
            "diagnóstico", "edad", "sexo", "peso", "talla", "preferencia", 
            "índice tabáquico", "tabaquismo", "tabaco", "alcohol", "drogas", 
            "comorbilidades", "antecedentes ginecológicos", "menarca", "embarazos", 
            "partos", "fum", "trh", "estado hormonal", "métodos anticonceptivos", 
            "cirugías", "originaria y residente", "seguridad social", "ocupación", "ahf", "g0 p0 c0 a0",
            "cáncer de", "resumen del"
        ]

        # Patrón para detectar fechas en formato DD.MM.YY (inicio de un nuevo registro)
        patron_fecha = re.compile(r'^\d{2}\.\d{2}\.\d{2}')

        texto_procesado = []
        clave_actual = None
        valor_actual = []

        for linea in self.informacion:
            linea = linea.strip()

            if not linea:
                continue

            # Detectar si la línea inicia con una palabra clave
            es_clave = any(linea.startswith(clave) for clave in claves) or re.match(patron_fecha, linea)
            
            if es_clave:
                # Si hay clave en curso, guardar lo acumulado
                if clave_actual:
                    texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")
                
                # Extraer clave y su valor si hay :
                if ':' in linea:
                    clave_actual, valor = map(str.strip, linea.split(':', 1))
                    valor_actual = [valor] if valor else []
                else:
                    clave_actual = linea
                    valor_actual = []
            elif clave_actual:
                # Continuar acumulando líneas bajo la clave actual
                valor_actual.append(linea)
            else:
                # Si no hay clave, solo agregar la línea
                texto_procesado.append(linea)

        # Guardar la última clave y su valor
        if clave_actual:
            texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")


        #eliminamos ":" al final de cada linea
        texto_procesado = [re.sub(r'[:\s]+$', '', linea) for linea in texto_procesado]

        print('\n'.join(texto_procesado))
        return '\n'.join(texto_procesado)

    def imprimir_datos(self):
        print('\n'.join(self.informacion))


limpiador = LimpiezaTexto('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido5.txt')
limpiador.preprocesar_texto()

mlaa / 314262 / 86 años / dr. soto
comorbilidades: hipertensión arterial sistémica (1995).
fum: 47 años
menarca a los 14 años. 2 embarazos, 2 partos.
métodos anticonceptivos desconoce.
cirugías: apendicectomía a los 20 años.
tabaquismo negado: exposición al humo de leña por 40 años por 2 horas.
ahf oncológicos: negados
originaria y residente: puebla
seguridad social: 1 (imss)
ocupación: hogar
cáncer de mama bilateral. mi cd! pt1c, pn1a, mo / md cli pt2, cno mo rh+ iher2-
resumen del padecimiento oncológico
04.08.23 po cirugía conservadora de mama bilateral + disección axilar izquierda: mama izquierda: carcinoma ductal infiltrante sin patrón específico, grado 3 (3+3+2), tamaño del tumor 2 x 1.5 x 1.9 cm, cdis presente, patrón cribiforme, sólido, grado 2, necrosis presente, ilv no identificado, ipn identificado, microcalcificaciones presentes en carcinoma invasor, mq negativos, gl evaluados 8, positivos 3. pt1c, pn1a ihq: re 90% +++, rp 90% +++, her2 negativo, ki67 30% mama derecha: carc

<>:88: SyntaxWarning: invalid escape sequence '\D'
<>:88: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_24236\175418400.py:88: SyntaxWarning: invalid escape sequence '\D'
  limpiador = LimpiezaTexto('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido5.txt')


'mlaa / 314262 / 86 años / dr. soto\ncomorbilidades: hipertensión arterial sistémica (1995).\nfum: 47 años\nmenarca a los 14 años. 2 embarazos, 2 partos.\nmétodos anticonceptivos desconoce.\ncirugías: apendicectomía a los 20 años.\ntabaquismo negado: exposición al humo de leña por 40 años por 2 horas.\nahf oncológicos: negados\noriginaria y residente: puebla\nseguridad social: 1 (imss)\nocupación: hogar\ncáncer de mama bilateral. mi cd! pt1c, pn1a, mo / md cli pt2, cno mo rh+ iher2-\nresumen del padecimiento oncológico\n04.08.23 po cirugía conservadora de mama bilateral + disección axilar izquierda: mama izquierda: carcinoma ductal infiltrante sin patrón específico, grado 3 (3+3+2), tamaño del tumor 2 x 1.5 x 1.9 cm, cdis presente, patrón cribiforme, sólido, grado 2, necrosis presente, ilv no identificado, ipn identificado, microcalcificaciones presentes en carcinoma invasor, mq negativos, gl evaluados 8, positivos 3. pt1c, pn1a ihq: re 90% +++, rp 90% +++, her2 negativo, ki67 30% mama

In [33]:
import json
import re
import os

class TextoJson:
    def __init__(self, nombreArchivo):
        self.limpiador = LimpiezaTexto(nombreArchivo)
        self.informacion = self.limpiador.preprocesar_texto().splitlines()
        # vectores
        self.data = {
            "antecedentes_personales": {},
            "antecedentes_familiares": {},
            "estadia_tumoral": {},
            "inmunohistoquimica_tumoral": {},
            "tipo": {},
        }

    def estructurar_data(self):
        return json.dumps(self.data, indent=4, ensure_ascii=False)
    
    def imprimir_data(self):
        print(json.dumps(self.data, indent=4, ensure_ascii=False))
    
    def extraer_datos_antecedentes_personales(self):
        # secciones de antecedentes_personales
        antecedentes_personales_secciones = {
            "edad": None,
            "sexo": "femenino",
            "peso": None,
            "talla": None,
            "preferencia": None,
            "índice_tabáquico": None,
            "alcohol": None,
            "drogas": None,
            "comorbilidades": None,
            "antecedentes_ginecológicos": {
                "fum": None,
                "menarca": None,
                "embarazos": None,
                "partos": None,
                "trh": None,
                "estado_hormonal": None,
                "métodos_anticonceptivos": None
            }
        }

        #palaras clave para antecedentes_personales que buscaremos en el texto
        antecedentes_personales_keywords = [
            "edad", "sexo", "peso", "talla", "preferencia", 
            "índice tabáquico", "tabaco", "tabaquismo", "alcohol", "drogas", 
            "comorbilidades", "antecedentes ginecológicos", "menarca", "embarazos", "partos", "fum", "trh", 
            "estado hormonal", "métodos anticonceptivos"
        ]

        texto_procesado = self.informacion
        isEdad = False

        # Agregar una regex para detectar edades en el formato "XX años"
        edad_regex = re.compile(
            r'\b(?:edad:?\s*(\d+)(?:\s*años)?)|(?:\b(\d+)\s*años(?:\s*de\s*edad)?)|^(\w+)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$',            re.IGNORECASE
        )
        for linea in texto_procesado:
            # Buscar una edad en el formato específico
            match = edad_regex.match(linea)
            if match:
                if match.group(1):  # "edad: XX" o "edad: XX años"
                    antecedentes_personales_secciones["edad"] = match.group(1).strip()
                elif match.group(2):  # "XX años" o "XX años de edad"
                    antecedentes_personales_secciones["edad"] = match.group(2).strip()
                elif match.group(5):  # Formato "gacmc / ... / XX años / ..."
                    antecedentes_personales_secciones["edad"] = match.group(5).strip()
                isEdad = True
            
            for keyword in antecedentes_personales_keywords:
                if keyword in linea:
                    #si encuentra la palabra "edad"
                    if keyword == "edad" and not isEdad:
                        if ":" in linea:
                            antecedentes_personales_secciones["edad"] = linea.split(":")[1].strip()
                        else:
                            antecedentes_personales_secciones["edad"] = linea.split(keyword)[1].strip()
                        isEdad = True
                    
                    #si la keyword es "tabaquismo" o "tabaco" o "índice tabáquico" y si tiene o no ":" que separan_
                    elif keyword == "tabaquismo" or keyword == "tabaco" or keyword == "índice tabáquico":
                        ##removemos los ":" 
                        if ":" in linea:
                            linea = linea.replace(":", "")
                            antecedentes_personales_secciones["índice_tabáquico"] = linea.split(keyword)[1].strip()
                        # if ":" in linea:
                        #     antecedentes_personales_secciones["índice_tabáquico"] = linea.split(":")[1].strip()
                        # else:
                        #     antecedentes_personales_secciones["índice_tabáquico"] = linea.split(keyword)[1].strip()

                    elif keyword in ["fum", "menarca", "trh", "estado hormonal", "métodos anticonceptivos", "embarazos", "partos"]:
                        # Verificar las keywords de antecedentes ginecológicos en la misma línea
                        pattern = re.compile(r'(fum|menarca|trh|estado hormonal|métodos anticonceptivos|embarazos|partos)', re.IGNORECASE)
                        matches = pattern.findall(linea)
                        if len(matches) > 1:
                           # Dividir la línea en partes basadas en varios separadores posibles
                            partes = re.split(r'[\/,\.]', linea)
                            for parte in partes:
                                # Asegurarte de que cada elemento sea una cadena válida
                                if isinstance(parte, str):  # Por seguridad, verifica si el elemento es una cadena
                                    parte = parte.strip()  # Elimina espacios al inicio y al final
                                    
                                for keyword in matches:
                                    if keyword in parte:

                                        if ":" in parte:
                                            antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(":")[1].strip()
                                            #self.asignar_si_vacio(antecedentes_personales_secciones["antecedentes_ginecológicos"], keyword.strip().lower(), parte.split(":")[1].strip())
                                        else:
                                            # Verificar si el valor precede a la clave
                                            if parte.strip().startswith(keyword):
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(keyword)[1].strip()
                                                #self.asignar_si_vacio(antecedentes_personales_secciones["antecedentes_ginecológicos"], keyword.strip().lower(), parte.split(match)[1].strip())
                                            else:
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(keyword)[0].strip()
                                                #self.asignar_si_vacio(antecedentes_personales_secciones["antecedentes_ginecológicos"], keyword.strip().lower(), parte.split(keyword)[0].strip())
                            continue
                        
                        else:
                            if ":" in linea:
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.replace(" ", "_")] = linea.split(":")[1].strip()
                            else:
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.replace(" ", "_")] = linea.split(keyword)[1].strip()
                    
                    ##tratamos las comoorbilidades dado que es una linea con varias comorbilidades separadas por "/"
                    elif keyword == "comorbilidades":
                        comorbilidades = linea
                        # Dividir la cadena en comorbilidades individuales
                        comorbilidades_list = re.split(r'\d+\.\s', comorbilidades)

                        # Eliminar posibles cadenas vacías resultantes de la división
                        comorbilidades_list = [comorbilidad for comorbilidad in comorbilidades_list if comorbilidad]

                        comorbilidades_dict = {}

                        for comorbilidad in comorbilidades_list:
                            comorbilidad = comorbilidad.strip()
                            # Extraer el año
                            match = re.search(r'\((\d{4})\)', comorbilidad)
                            if match:
                                year = match.group(1)
                                # Separar nombre y descripción
                                parts = comorbilidad.split(f"({year})")
                                nombre = parts[0].strip()
                                descripcion = parts[1].strip() if len(parts) > 1 else ""

                                # Si la descripción es solo un punto, lo agregamos al nombre
                                if descripcion == ".":
                                    descripcion = None  # Dejamos la descripción vacía
                                    nombre = nombre.rstrip('.')  # Quitamos el punto del nombre

                                if year not in comorbilidades_dict:
                                    comorbilidades_dict[year] = []

                                comorbilidades_dict[year].append({
                                    "nombre": nombre.replace("comorbilidades: ", "").strip(),
                                    "descripción": descripcion.lstrip(': ') if descripcion else None
                                })
                        antecedentes_personales_secciones["comorbilidades"] = comorbilidades_dict

                    else:
                        if keyword == "edad" and isEdad == True:
                            continue
                        elif ":" in linea:
                            antecedentes_personales_secciones[keyword.replace(" ", "_")] = linea.split(":")[1].strip()
                        
                        else:
                            antecedentes_personales_secciones[keyword.replace(" ", "_")] = linea.split(keyword)[1].strip()

        self.data["antecedentes_personales"] = antecedentes_personales_secciones
        #self.estructurar_data()     
        self.imprimir_data()
    

procesador = TextoJson('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido5.txt')
procesador.extraer_datos_antecedentes_personales()

mlaa / 314262 / 86 años / dr. soto
comorbilidades: hipertensión arterial sistémica (1995).
fum: 47 años
menarca a los 14 años. 2 embarazos, 2 partos.
métodos anticonceptivos desconoce.
cirugías: apendicectomía a los 20 años.
tabaquismo negado: exposición al humo de leña por 40 años por 2 horas.
ahf oncológicos: negados
originaria y residente: puebla
seguridad social: 1 (imss)
ocupación: hogar
cáncer de mama bilateral. mi cd! pt1c, pn1a, mo / md cli pt2, cno mo rh+ iher2-
resumen del padecimiento oncológico
04.08.23 po cirugía conservadora de mama bilateral + disección axilar izquierda: mama izquierda: carcinoma ductal infiltrante sin patrón específico, grado 3 (3+3+2), tamaño del tumor 2 x 1.5 x 1.9 cm, cdis presente, patrón cribiforme, sólido, grado 2, necrosis presente, ilv no identificado, ipn identificado, microcalcificaciones presentes en carcinoma invasor, mq negativos, gl evaluados 8, positivos 3. pt1c, pn1a ihq: re 90% +++, rp 90% +++, her2 negativo, ki67 30% mama derecha: carc

<>:179: SyntaxWarning: invalid escape sequence '\D'
<>:179: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_24236\858281458.py:179: SyntaxWarning: invalid escape sequence '\D'
  procesador = TextoJson('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido5.txt')
